In [16]:
from datetime import datetime
from subgrounds.subgraph import SyntheticField, FieldPath
from subgrounds.subgrounds import Subgrounds
import pandas as pd

In [17]:
sg = Subgrounds()
snapshot = sg.load_api('https://hub.snapshot.org/graphql')

In [18]:
snapshot.Proposal.datetime = SyntheticField(
  lambda timestamp: str(datetime.fromtimestamp(timestamp)),
  SyntheticField.STRING,
  snapshot.Proposal.end,
)

In [19]:
proposals = snapshot.Query.proposals(
  orderBy='created',
  orderDirection='desc',
  first=10000,
  where=[
    snapshot.Proposal.space == 'olympusdao.eth',
    snapshot.Proposal.state == 'closed',
  ]
)

In [20]:
proposals_snapshots = sg.query_df([
  proposals.title,
  proposals.id,
  proposals.body
])

In [21]:
proposals_choices = sg.query(proposals.choices)

In [22]:
proposals_choices = pd.DataFrame(proposals_choices, columns = ['option_a', 'option_b', 'option_c', 'option_d', 'option_e','option_f','option_g'])

In [23]:
olympus_governance_view = pd.concat([proposals_snapshots,proposals_choices], axis=1)

In [24]:
##let's view the output just to make sure
olympus_governance_view

,proposals_title,proposals_id,proposals_body,option_a,option_b,option_c,option_d,option_e,option_f,option_g
0,OIP-94: Interim Ranged Stability Policy Levers,0x6413984c8f1076e3f46f196810ba51781f25f4d2f1eb...,abipup\n \n 2 days ago\n\nSumma...,Inverse bonds & adjust liquidity,Do nothing,None,None,None,None,None
1,OIP-93: Mint and Sync,0xf0e62d0a9eeb47a295bab9a8d0cc84c0d4de0a8cb000...,Summary:\n\nEnable minting into liquidity pool...,"1. Implement, no change to RR","2. Implement, change RR",3. Do not implement,None,None,None,None
2,TAP-12 Pilot Deposit on Tribe Turbo,0x7d191f4a7f2ea3d138c07f73d568b93c96fc5ffce267...,Summary\n\nSeek authority from the community t...,Approve Pilot Deposit,Do not approve pilot deposit,None,None,None,None,None
3,TAP-11 Ribbon Finance Whitelist,0xe52a4af6307bea40a34f2eaf9a93f11c8abf382020cc...,Summary: Seek authority from the community to ...,Allow Ribbon Finance,Do not allow Ribbon Finance,None,None,None,None,None
4,OIP - 92 : Whitelist FEI as a treasury Reserve...,0xe9aa47f39bc5ace6ad5a19c08007a3683386e19c1390...,Summary\n\nWhitelist FEI as our next reserve a...,Approve FEI in the Treasury,Do not approve FEI in Treasury,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...
192,OIP-5: Lower LP Staking Rewards,QmTrwGU99RMqTeeCKy5oW9Le9M2mXHHdNeEWzn7T67watz,Summary: We should decrease LP staking rewards...,For: Option 1,For: Option 2,Against,None,None,None,None
193,Deposit SLP,QmZhSNWAzrDy5KiGSeBsKBXUHW6D4vJnfCuqmdai8LPETd,Summary: The DAO currently holds liquidity tha...,Deposit; keep Onsen,Deposit; no Onsen,Don't deposit,None,None,None,None
194,Loosen Policy,QmZJb2evEjsxtLz92XBuhApWty69fVqUpFHzuVjMjCd3Se,Summary: Let's adjust system parameters away f...,Do it,Don't do it,None,None,None,None,None
195,Add DAI Bonds,QmUC5xa4YaWZBV7fy241svBQ8bHSMksm8kB7avk1WyWv9m,Summary: We can launch bonds using DAI instead...,Add; remove sales,Add; phase out sales,Add; keep sales,Do not add,None,None,None


In [25]:
total_proposals = len(olympus_governance_view)
total_proposals

197

In [27]:
   proposal_id = olympus_governance_view.iloc[2,1]

In [28]:
print(proposal_id)

0x7d191f4a7f2ea3d138c07f73d568b93c96fc5ffce2674271fbced88406d26fa7


In [29]:

    vote_tracker = snapshot.Query.votes(
    orderBy = 'created',
    orderDirection='desc',
    first=10000,
    where=[
      snapshot.Vote.proposal == proposal_id
    ]
    )

In [30]:
voting_snapshots = sg.query_df([
    vote_tracker.id,
    vote_tracker.voter,
    vote_tracker.created,
    vote_tracker.choice
])

In [31]:
voting_snapshots

,votes_id,votes_voter,votes_created,votes_choice
0,0xa4ffd5df54c140131b3cdcc6cc2c1cbe3c9dab64f7dd...,0xEb4Db23c27253077Fb3080adda6C5C127b0dACAe,1650641362,1
1,0x705504d69e7063b58d2268f211df9eb886011585d2ec...,0x538e597a31F044cD0d88afA914F2602D08cDB7f0,1650639909,1
2,0x6ac0b612d8ecb16db30c1486bb609a53b9e88c9ac37e...,0x6C89c435082fec970f4a04B25Fc972c95dEc05Bb,1650638315,1
3,0xd673cf52732dd83879513141672eb966f477e0961f49...,0x306ca959737a347d3639D7Ae0cC87FAA6Cd1884b,1650638179,1
4,QmawnyWbYBbC4YmXsYmNipGQdBMF2TEM7VwBGaeSjtPux4,0xb83E4b2471f54979e5C276a4b453E44193436523,1650638121,1
...,...,...,...,...
149,0x23cbf349a9e8c9ccbea0aaf7597b2c28221f689c24cd...,0x0bD43c6cD201697E88Aa795B3bb79E7dFBEED8EE,1650462811,1
150,0x9aecaad9348066e5560c97acbb87087c3e0796a25662...,0x23d5F49222BdB9FD5944b9e8ce0B5B21c4079C64,1650462749,1
151,0xd1bfa254a0f6c81bce7668ca91dab9cd4ce0c22a33a2...,0x0561a78021D8966ddD20c28C6c4318D8675eE1F0,1650462179,1
152,0x3baf37b72615ab6fd94a10138ad7e6f828479a11e2fc...,0x6487d336bDec94d4714a4d217B15aa5c7b9dfA99,1650461919,1


In [33]:
voting_snapshots['Proposal'] = proposal_id
print(voting_snapshots)

                                              votes_id  \
0    0xa4ffd5df54c140131b3cdcc6cc2c1cbe3c9dab64f7dd...   
1    0x705504d69e7063b58d2268f211df9eb886011585d2ec...   
2    0x6ac0b612d8ecb16db30c1486bb609a53b9e88c9ac37e...   
3    0xd673cf52732dd83879513141672eb966f477e0961f49...   
4       QmawnyWbYBbC4YmXsYmNipGQdBMF2TEM7VwBGaeSjtPux4   
..                                                 ...   
149  0x23cbf349a9e8c9ccbea0aaf7597b2c28221f689c24cd...   
150  0x9aecaad9348066e5560c97acbb87087c3e0796a25662...   
151  0xd1bfa254a0f6c81bce7668ca91dab9cd4ce0c22a33a2...   
152  0x3baf37b72615ab6fd94a10138ad7e6f828479a11e2fc...   
153  0xb9637484ca4655c1d0d8370e9d18024ba518ae956398...   

                                    votes_voter  votes_created  votes_choice  \
0    0xEb4Db23c27253077Fb3080adda6C5C127b0dACAe     1650641362             1   
1    0x538e597a31F044cD0d88afA914F2602D08cDB7f0     1650639909             1   
2    0x6C89c435082fec970f4a04B25Fc972c95dEc05Bb     1650638315 